In [1]:
## headings
"""
made by weiyw @ 2019-04-04
made to use both vz and vx components
"""
import os
import time
import struct
import numpy as np
from glob import glob

import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

import keras
from keras.models import Model
from keras.layers import Conv2D#Convolution2D
# from keras.layers.convolutional import Conv3D
from keras.layers.normalization import BatchNormalization
from keras.layers import (
    Input,
    Activation,
    merge,
    Dense,
    Reshape
)
import metrics as metrics


Using TensorFlow backend.


In [2]:
def get_min_nonzero_value(data_in, init_min = 1):
    min_value = init_min
    for bb in data_in:
        for aa in bb:
            if abs(aa) < min_value and aa != 0:
                min_value = abs(aa)
    return min_value


In [3]:
def get_bindata(data_path, data_index, data_phase, nt, nr):
    # data_index start from 1
    model_num = int( ( int(data_index) - 1 ) / 51) + 1
    source_num = ( int(data_index) - 1 ) % 51 + 1
    data_name = data_path + "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
#     print(data_name)
    
    out_data = np.empty((nt,nr))
    FA = open(data_name, "rb")
    FA.seek(3232,0)
    for tt in range(nt):
        for rr in range(nr):
            data = FA.read(4)
            data_float = struct.unpack("f", data)[0]
            out_data[tt][rr] = data_float
     #np.shape(out_data)   # (4000,301)
#     cut_data = out_data[0:4000, 8:401]
    return out_data  


In [4]:
class Dataloader():
    def __init__(self, data_path, nt, nr, nph):
        self.data_path = data_path
        self.nt = nt
        self.nr = nr
        self.nph = nph
    
    
    def readin_bin_data(self, data_name):
        out_data = np.empty((self.nt, self.nr))
        FA = open(data_name, "rb")
        FA.seek(3232,0)
        for tt in range(self.nt):
            for rr in range(self.nr):
                data = FA.read(4)
                data_float = struct.unpack("f", data)[0]
                out_data[tt][rr] = data_float
         #np.shape(out_data)   # (4000,301)
    #     cut_data = out_data[0:4000, 8:401]
        return out_data  
    
    
#     def load_data(self, batch_size=1, is_testing=False):
#         data_type = "train" if not is_testing else "test"
# #         path = os.path.join(self.data_path, self.dataset_name)
        
#         return x_data, y_data
    
    
    def load_batch(self, batch_size=1, is_testing=False, ratio=0.5):
        data_type = "train" if not is_testing else "test"
#         path = glob('%s/%s/%s/*' % (self.data_path, self.dataset_name, data_type))
        path = glob('%s/vz/*' % self.data_path)
#         path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))
#         path = os.path.join(self.data_path, self.dataset_name)
        
        self.n_batches = int( len(path) / batch_size * ratio )
#         self.n_batches = 1
        
#         while True:
        for i in range(self.n_batches):
#             batch = path[ i * batch_size : (i+1) * batch_size]
#             batch_num = [ i * batch_size : (i+1) * batch_size]
#             x_data, y_data = [], [] ## 用list还是array？
            x_data = np.empty((batch_size, self.nt, self.nr, self.nph)) ## b-4000-300-2, vz, vx
            y_data = np.empty((batch_size, self.nt, self.nr, self.nph)) ## b-4000-300-2, div, curl
            for data_index_local in range(batch_size): ## 0 ~ batchsize-1
                data_index = i * batch_size + data_index_local + 1
                model_num = int( ( int(data_index) - 1 ) / 51) + 1
                source_num = ( int(data_index) - 1 ) % 51 + 1
                if self.nph == 2:
                    for data_phase in ['vz', 'vx']:
                        p_flag = int(data_phase == 'vx')
                        data_name = self.data_path + "/" + data_phase + \
                        "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
#                         print(data_name)
                        x_data[data_index_local, :, :, p_flag] = self.readin_bin_data(data_name)
                    for data_phase in ['div', 'curl']:
                        p_flag = int(data_phase == 'curl')
                        data_name = self.data_path + "/" + data_phase + \
                        "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
#                         print(data_name)
                        y_data[data_index_local, :, :, p_flag] = self.readin_bin_data(data_name)
                if self.nph == 1:
                    data_phase = 'vz'
                    data_name = self.data_path + "/" + data_phase + \
                    "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
                    x_data[data_index_local, :, :, 0] = self.readin_bin_data(data_name)
                    data_phase = 'div'
                    data_name = self.data_path + "/" + data_phase + \
                    "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
                    y_data[data_index_local, :, :, 0] = self.readin_bin_data(data_name)                    
                yield x_data, y_data
                

In [5]:
def simpleNet(nt, nr, nph):
    x_input = Input( shape=( nt, nr, nph) )##one_piece
    conv1 = Conv2D(
        nb_filter=64, nb_row=3, nb_col=3, border_mode="same", data_format="channels_last")(x_input)
    conv1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                               beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
                               moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
                               beta_constraint=None, gamma_constraint=None)(conv1)
    conv2 = Conv2D(
        nb_filter=2, nb_row=3, nb_col=3, border_mode="same", data_format="channels_last")(conv1)
    # conv1 = Conv3D(filters=51, kernel_size=(2,2,2), strides=(1,1,1), 
    #                             padding='same', data_format="channels_last", dilation_rate=(1,1,1),
    #                            activation=None, use_bias=True, kernel_initializer='glorot_uniform',
    #                            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
    #                            activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(x_input)
    # conv1 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
    #                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
    #                            moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
    #                            beta_constraint=None, gamma_constraint=None)(conv1)
    # conv2 = Conv3D(filters=51, kernel_size=(2,2,2), strides=(1,1,1), 
    #                             padding='same', data_format="channels_last", dilation_rate=(1,1,1),
    #                            activation=None, use_bias=True, kernel_initializer='glorot_uniform',
    #                            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
    #                            activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(conv1)

    # conv2 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
    #                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
    #                            moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
    #                            beta_constraint=None, gamma_constraint=None)(conv2)

    # conv3 = Conv3D(filters=51, kernel_size=(2,2,2), strides=(1,1,1), 
    #                             padding='same', data_format="channels_last", dilation_rate=(1,1,1),
    #                            activation=None, use_bias=True, kernel_initializer='glorot_uniform',
    #                            bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None,
    #                            activity_regularizer=None, kernel_constraint=None, bias_constraint=None)(conv2)
    # conv3 = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
    #                            beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', 
    #                            moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, 
    #                            beta_constraint=None, gamma_constraint=None)(conv3)

    # convLSTM_1 = keras.layers.ConvLSTM2D(filters=51, kernel_size=(2,2), strides=(1, 1), padding='valid', 
    #                                      data_format='channels_last', dilation_rate=(1, 1), activation='tanh', 
    #                                      recurrent_activation='hard_sigmoid', use_bias=True,
    #                                      kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
    #                                      bias_initializer='zeros', unit_forget_bias=True, 
    #                                      kernel_regularizer=None, recurrent_regularizer=None, 
    #                                      bias_regularizer=None, activity_regularizer=None, 
    #                                      kernel_constraint=None, recurrent_constraint=None, 
    #                                      bias_constraint=None, return_sequences=True, 
    #                                      go_backwards=False, stateful=False, dropout=0.0, recurrent_dropout=0.0)(conv3)


    model = Model(inputs=x_input, outputs=conv2)
    # model.summary()
    return model


In [6]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [ ]:
data_path = "/media/wywdisk/VSPdata/data/haveinvx/layer2_haveinvx"

nt = 4000     # time step
nr = 400      # receiver
ns = 51       # shot
nmodel = 2    # batch
nph = 2

## training parameters
TPepoches = 4
TPbatch_size = 4


## setup
model = simpleNet(nt, nr, nph)
my_data_loader = Dataloader(data_path, nt, nr, nph)
model.compile(loss='mean_squared_error', optimizer=Adam(lr=lr_schedule(0)), metrics=['accuracy'])

In [ ]:
# # Score trained model.
# ts = time.time()
# scores = model.evaluate(x_valid, y_valid, batch_size=None, steps=1, verbose=1)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

# # print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
# #         (score[0], score[1], score[1] * (mx - mn) / 2.))
# print("\nelapsed time (eval cont): %.3f seconds\n" % (time.time() - ts))

In [ ]:
## prdiction
# y_predict = model.predict(x_test, batch_size=None, steps=1, verbose = 1)
save_dir = os.path.join(os.getcwd(), 'saved_models')
model.load_weights(os.path.join( save_dir, '{}.best.h5'.format(hyperparams_name)))
predict = model.predict_generator(my_data_loader.load_batch(batch_size=TPbatch_size, is_testing=True, ratio=0.1),\
                                  steps=1, max_queue_size=10, \
                                  workers=1, use_multiprocessing=False, verbose=0)

In [ ]:
## check for predicted data
a1 = y_predict
print("Information of a1 :")
print("The shape of a1   :",a1.shape)
print("The range        : ", [a1.min(),a1.max()])

In [ ]:
## plot y_predict
from matplotlib.ticker import MultipleLocator
lines = 1
cols = 2
forts = 12
ss1 = 300
ss2 = 400

x_libs = [0,0, 500, 1000, 1500, 2000]#np.linspace(0,1500,4)#[0, 500, 1000, 1500]
y_libs = np.linspace(0,2,9)#[0,0.4, 0.8, 1.2, 1.6, 2]

fig, axs = plt.subplots(1,2,figsize=(10, 4), sharey=True)

# obj = 'y_predict'
# im = plt.imshow(eval(obj)[0,:,:,1], extent=[0, 512, 512,0])
# plt.colorbar(im)

# ax1 = plt.subplot(131)
ax1 = plt.subplot(lines,cols,1)
obj = 'y_predict'
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(eval(obj)[0,:,:,0], extent=[0, ss1, ss2,0])
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("predicted P-wave", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
# ax1 = fig.add_subplot(131)
ax1.xaxis.set_major_locator(MultipleLocator(75))
ax1.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax1.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax1.get_xticklabels(), fontsize=10);
# plt.tight_layout()

# ax1 = plt.subplot(131)
ax2 = plt.subplot(lines,cols,2)
obj = 'y_predict'
# print(obj, ": ", eval(obj).shape, "\t", [eval(obj).min(), eval(obj).max()] )
im = plt.imshow(eval(obj)[0,:,:,1], extent=[0, ss1, ss2,0])
plt.colorbar(im)
# plt.title("The elastic z component of VSP data", fontsize=forts);
plt.title("predicted S-wave", fontsize=forts);
plt.xlabel('depth(m)', fontsize=forts);
plt.ylabel('time(s)', fontsize=forts);
# ax1 = fig.add_subplot(131)
ax2.xaxis.set_major_locator(MultipleLocator(75))
ax2.xaxis.set(ticklabels=x_libs)#[0,300,600, 900, 1200, 1500]);
ax2.yaxis.set(ticklabels=y_libs)#[0,0.4, 0.8, 1.2, 1.6, 2]);
plt.setp(ax2.get_xticklabels(), fontsize=10);
# plt.tight_layout()

